# The fast Fourier transform

We consider a question: ***How to Efficiently compute multiplication of polunomial***?
i.e.
> For two given degree-$d$ polynomial $A(x)=\sum^d_{i=0}a_ix^i, B(x)=\sum^d_{i=0}b_ix^i$, we want to compute the product of them:
> $$
C(x)=A(x)\cdot B(x)=\sum^{2d}_{i=0}c_ix^i\\
where\ c_i=\sum_{j=0}^ia_jb_{i-j}
> $$

Obviously there is a fact that:
> A degree-$d$ polynomial $A(x)=\sum^d_{i=0}a_ix^i$ is determinated by $d+1$ values:
> 1. Its coefficients:
> $$
a_0,a_1,\dots a_d
> $$
> 2. Its distinct n (argument,value)-pairs
> $$
> (x_0, A(x_0)),\dots, (x_{d+1},A(x_{d+1}))
> $$

If we calculate the coefficients of $C(x)$.  
i.e. by the equation $c_i=\sum_{j=0}^ia_jb_{i-j}$, then the time complexity is $O(m^2)$. 


There is a another viewpoint to consider the above two things:
Note that:
$$
\begin{bmatrix}
A(x_0)\\
A(x_1)\\
\dots\\
A(x_d)
\end{bmatrix}=
\begin{bmatrix}
&x_0^0 &x_0^1 &\dots &x_0^d\\
&x_1^0 &x_1^1 &\dots &x_1^d\\
&\dots &\dots &\dots &\dots\\
&x_d^0 &x_d^1 &\dots &x_d^d\\
\end{bmatrix}
\begin{bmatrix}
a_0\\
a_1\\
\dots\\
a_d
\end{bmatrix}
$$
If the $d+1$ variables $x_i$ are distinct, then the matrix in middle(we express as $M$) is invertible, so we can multiple $M/M^{-1}$ to transfer between coefficients and values.

The matrix $M$ we called **Vandermonde Matrix**.

In [9]:
import cmath

In [10]:
class Polynomial:
    def __init__(self, coefficients: list) -> None:
        coefficients = coefficients.copy()
        while len(coefficients) > 1 and coefficients[-1] == 0:
            coefficients.pop()
        self._coefficients = coefficients# From the constant item to the heightest degree
        self.degree = len(coefficients) - 1
        
    def __call__(self, x: int | float | complex)-> int | float | complex:
        return self._value_of(x)
    
    def _value_of(self, x: int | float | complex)-> int | float | complex:
        result = 0
        for i, c in enumerate(self._coefficients):
            result += c * (x ** i)
        return result
    
    def __bool__(self)-> bool:
        return any(c != 0 for c in self._coefficients)
    
    def __eq__(self, value: object) -> bool:
        if not isinstance(value, Polynomial):
            return False
        return self._coefficients == value.get_coefficients()
    
    def __str__(self) -> str:
        result = ""
        for i,c in enumerate(self._coefficients):
            if i == 0: 
                result = str(c)
            elif i == 1:
                result += f" + {c}x"
            else:
               result += f" + {c}x^{i}"
        return result
    
    def __repr__(self) -> str:
        return f"Polynomial({self._coefficients})"
    
    def __len__(self) -> int:
        return self.degree + 1
    
    def __add__(self, other: object)-> 'Polynomial':
        if isinstance(other, (int, float, complex)):
            newCoeffs = self._coefficients.copy()
            newCoeffs[0] += other
            return Polynomial(newCoeffs)
        elif isinstance(other, Polynomial):
            newDegree = max(self.degree, other.degree)
            newCoeffs = []
            for i in range(newDegree + 1):
                tem = 0.0
                if i < len(self._coefficients):
                    tem += self._coefficients[i]
                if i < len(other.get_coefficients()):
                    tem += other.get_coefficients()[i]
                newCoeffs.append(tem)
            return Polynomial(newCoeffs)
        else:
            return NotImplemented
        
    def __radd__(self, other: object)-> 'Polynomial':
        return self + other
    
    def __sub__(self, other: object)-> 'Polynomial':
        if isinstance(other, (int, float, complex)):
            newCoeffs = self._coefficients.copy()
            newCoeffs[0] -= other
            return Polynomial(newCoeffs)
        elif isinstance(other, Polynomial):
            newDegree = max(self.degree, other.degree)
            newCoeffs = []
            for i in range(newDegree + 1):
                tem = 0.0
                if i < len(self._coefficients):
                    tem += self._coefficients[i]
                if i < len(other.get_coefficients()):
                    tem -= other.get_coefficients()[i]
                newCoeffs.append(tem)
            return Polynomial(newCoeffs)
        else:
            return NotImplemented
        
    def __neg__(self) -> 'Polynomial':
        newCoeff = [-self._coefficients[i] for i in range(self.degree + 1)]
        return Polynomial(newCoeff)
        
    def __rsub__(self, other: object)-> 'Polynomial':
        return - (self - other)
        
    
    def __mul__(self, other: object)-> 'Polynomial':
        if isinstance(other, (int, float, complex)):
            newCoeffs = [self._coefficients[i] * other for i in range(self.degree + 1)]
            return Polynomial(newCoeffs)
        elif isinstance(other, Polynomial):
            newDegree = self.degree + other.degree
            newCoeffs = []
            for i in range(newDegree + 1):
                tem = 0.0
                for j in range(i + 1):
                    if (j > self.degree) or (i-j > other.degree):
                        continue
                    tem += self._coefficients[j] * other.get_coefficients()[i-j]
                newCoeffs.append(tem)
            return Polynomial(newCoeffs)
        else:
            return NotImplemented
        
    def __rmul__(self, other: object)-> 'Polynomial':
        return self * other
    
    def __pow__(self, power: int)-> 'Polynomial':
        if not isinstance(power, int) or power < 0:
            return NotImplemented
        if power == 0:
            return Polynomial([1])
        result = Polynomial([1])
        base = self
        while power > 0:
            if power % 2 == 1:
                result = result * base
            base = base * base
            power //= 2
        return result
    
    def get_coefficients(self) -> list[complex]:
        return self._coefficients.copy()

But what if we condiser the values of $A(x)$ and $B(x)$?
## Compute the products of values!

Its seems more simple. We need 4 steps to product two polynomial:
+ Polunomial Multiplication
  + Input: Coefficients of two polynomials, $A(x)$ and $B(x)$, of degree $d$
  + Output: Their product $C=A·B$
  + **Selection**
    + Pick some points $x_0,x_1,...,x_{n−1}$, where $n≥2d+1$
  + **Evaluation**
    + Compute $A(x_0),A(x_1),...,A(x_{n−1})$ and $B(x_0),B(x_1),...,B(x_{n−1})$
  + **Multiplication**
    + Compute $C(x_k) = A(x_k)B(x_k)$ for all $k=0,...,n−1$
  + **Interpolation**
    + Recover $C(x) = c_0 + c_1x+···+ c_2^dx_2^d$

We skip the selection, analyse the evaluation first.
## 1. Evaluation
When we directly compute the value of each point of $A(x)=\sum^d_{i=0}a_ix^i$, it's a $O(d)$ scale problem, that we need compute all  $d$ items of the polynomial. But consider the following case:
> If we choose two point $t$ and $-t$, then we have
> $$
\begin{aligned}
&A(t) = \sum^d_{i=0}a_it^i\\
&A(-t) = \sum^d_{i=0}a_i(-t)^i = \sum^d_{\text{i is even}}a_it^i - \sum^d_{\text{i is odd}}a_it^i\\
\end{aligned}
> $$ 

More generally, we define that:
$$
A(x) = A_{e}(x^2) + xA_o(x^2)
$$
where $A_e$ donates the even-numbered coeffcients(e.g. $a_0, a_2,\dots$), $A_o$ donates the odd-numbered coeffcients.

We divide the problem into two half subproblem, with linearly time! 
i.e. 
$$
T(n) = 2T(\frac{n}{2})+O(n)
$$

But obviously there is a **BUG**! Note that $A_{e/o}(x^2)$'s argument is $x^2$, thus we can't directly find a rational variable that its square is $-x^2$. So to make the recurse process continue, we consider to expend the domain to $\mathbb{C}$!

Consider we use **complex n-th roots of unity** $w$, where $w=e^{2\frac{\pi}{n}i}$. So that we have the following properties:
$$
\begin{align}
&1,w,w^2,\dots w^{n-1}\text{ are distinct!}\\
&w^n=1\\
\end{align}
$$

Note that under module $n$, $\{0,1,\dots n-1\}\rightarrow^{\times 2}\{0,2,\dots\}$. We assume that $n$ is even, then each root $w^i$ has a unique coordinate root $w^j$, where $2i\equiv \pmod{n}$, i.e. $(w^i)^2 = (w^j)^2$! Then we have efficient finish the **sellection** and **evaluation** process!
We have:
$$
\begin{aligned}
&j = i + \frac{n}{2} \pmod{n}\\
&A(w^i) = A_e(w^{2i}) + w^iA_o(w^{2i})\\
&A(w^j) = A_e(w^{2j}) + w^jA_o(w^{2j}) = A_e(w^{2i}) + w^jA_o(w^{2i})\\
\end{aligned}
$$

In [11]:
def divideCoeffs(coeffs: list[complex])->list[list[complex]]:
    coeffAe = coeffs[0::2]
    coeffAo = coeffs[1::2]
    if not coeffAe:
        coeffAe = [0j]
    if not coeffAo:
        coeffAo = [0j]
    return [coeffAo, coeffAe]

def generateNthRoots(n: int)->list[complex]:
    return [cmath.exp(2j * cmath.pi * k / n) for k in range(n)]

def evaluate(p:'Polynomial', n)->list[complex]:
    if n & (n-1) != 0:
        n = 1 << (n-1).bit_length()
    roots = generateNthRoots(n)
    coeffs = p.get_coefficients()
    for _ in range(n-len(p)):
        coeffs.append(complex(0))
    return FFT(coeffs, roots)

def FFT(coeffs: list[complex], roots: list[complex])-> list[complex]:
    n = len(roots)
    if n == 1:
        return [coeffs[0]]
    
    #n must be power of 2
    tem = n
    while tem > 1:
        if tem%2 == 1:
            raise NotImplementedError
        tem = tem//2
    
    Ao, Ae = divideCoeffs(coeffs)
    result = [complex(0) for _ in range(n)]
    subroots = roots[::2]
    resultAo = FFT(Ao, subroots)
    resultAe = FFT(Ae, subroots)
    for i in range(n//2):
        result[i] = resultAe[i] + roots[i] * resultAo[i]
        result[i + n//2] = resultAe[i] - roots[i] * resultAo[i]
    return result
        
        

The next problem is **How to use the n (argument, value)-pairs to recover the function**?

# 2. Interpolation

The **interpolation** is essentially the process of **recover the polynomial from values to coefficient**. Consider the **Vandermonde Matrix** expression of polynomial:
$$
\begin{bmatrix}
A(x_0)\\
A(x_1)\\
\dots\\
A(x_d)
\end{bmatrix}=
\begin{bmatrix}
&x_0^0 &x_0^1 &\dots &x_0^d\\
&x_1^0 &x_1^1 &\dots &x_1^d\\
&\dots &\dots &\dots &\dots\\
&x_d^0 &x_d^1 &\dots &x_d^d\\
\end{bmatrix}
\begin{bmatrix}
a_0\\
a_1\\
\dots\\
a_d
\end{bmatrix}=
\begin{bmatrix}
&1 &1 &\dots &1\\
&w_d^{1\times 0} &w_d^{1\times 1} &\dots &w_d^{1\times d}\\
&\dots &\dots &\dots &\dots\\
&w_d^{d\times 0} &w_d^{d\times 1} &\dots &w_d^{d\times d}\\
\end{bmatrix}
\begin{bmatrix}
a_0\\
a_1\\
\dots\\
a_d
\end{bmatrix}
$$
The process of computing the $d+1$ values of polynomial can be seen as a matrix multiplication. It's actually **Fourier Transform**(In my another note) if the $d+1$ argument is the n-th unit roots. The algorithm we use to speed up the evaluation is essectially one of the **Fast Foutier Transform**, we donate as 
$$
\{A(x_i)\}=FFT(\{a_i\}, w_d)
$$
, then the inverse process of recover the coefficients can also be seen as a matrix multiplication:
$$
\begin{bmatrix}
a_0\\
a_1\\
\dots\\
a_d
\end{bmatrix}=
\begin{bmatrix}
&1 &1 &\dots &1\\
&w_d^{1\times 0} &w_d^{1\times 1} &\dots &w_d^{1\times d}\\
&\dots &\dots &\dots &\dots\\
&w_d^{d\times 0} &w_d^{d\times 1} &\dots &w_d^{d\times d}\\
\end{bmatrix}^{-1}
\begin{bmatrix}
A(x_0)\\
A(x_1)\\
\dots\\
A(x_d)
\end{bmatrix}
$$

Note that :
$$
M_d(w_d)=
\begin{bmatrix}
&1 &1 &\dots &1\\
&w_d^{1\times 0} &w_d^{1\times 1} &\dots &w_d^{1\times d}\\
&\dots &\dots &\dots &\dots\\
&w_d^{d\times 0} &w_d^{d\times 1} &\dots &w_d^{d\times d}\\
\end{bmatrix}^{-1}=
\begin{bmatrix}
&1 &1 &\dots &1\\
&w_d^{-1\times 0} &w_d^{-1\times 1} &\dots &w_d^{-1\times d}\\
&\dots &\dots &\dots &\dots\\
&w_d^{-d\times 0} &w_d^{-d\times 1} &\dots &w_d^{-d\times d}\\
\end{bmatrix}^{-1}
=\frac{1}{d}M_d(w_d^{-1})\\
And\ w_d^{-1} = w_d^{d-1}
$$

i.e. we can use the inverse FFT to implement the interpolation:
$$
\{a_i\}=FFT^{-1}(\{A(x_i)\}, w_d)=\frac{1}{n}FFT(\{A(x_i)\}, w_d^{-1})
$$


In [12]:
def interpolate(n: int, values: list[complex])->list[complex]:
    old_roots = generateNthRoots(n)
    roots = [complex(1)]
    for i in range(1,n):
        roots.append(old_roots[n-i])
    return [coeff / n for coeff in FFT(values, roots)]

def quickly_multiplication_of_polynomial(p1: "Polynomial", p2: "Polynomial") -> Polynomial:
    n = len(p1) + len(p2) - 1
    if n & (n-1) != 0:
        n = 1 << (n-1).bit_length()
    values1 = evaluate(p1, n)
    values2 = evaluate(p2, n)
    values = [values1[i] * values2[i] for i in range(n)]
    return Polynomial(interpolate(n, values))
    
    
p = Polynomial([1,1])
print(p)
print(quickly_multiplication_of_polynomial(p,p))

1 + 1x
(1+5.551115123125783e-17j) + (2-2.392081711033608e-16j)x + (1-5.551115123125783e-17j)x^2 + 2.392081711033608e-16jx^3
